This example loads some data form medline_wos database through an
SQL query to a SQLite database.

Note that the query contains multiple filters for bona-fide research
publications, which includes a dismissial of journals with many reviews.
The query also removes cases where ut (web of science) and pmid (pubmed)
is ambiguous.

Here amount of authors and citations are queried.

Note that medline_wos is supposed to be static. Thus a local on-disk
cache with a capacity of 7 days is used to speed up repeated queries.

In addition some SQLite specific sytnax is used (e.g.: defining 
pubmed2taxon, which will lead to largest filtering on left to speed
up outer loop), (and writing JOINS as WHERE for better readibiliy)

In [1]:
from access_literature_data import medline

In [2]:
sql ='''
SELECT DISTINCT medline.pubmed_id, medline.pubdate_year, medline.amount_of_authors, citations.* 
FROM pubmed2taxon, medline, ut2pmid, citations
    WHERE
            pubmed2taxon.pubmed_id = medline.pubmed_id
        AND
            pubmed2taxon.pubmed_id = ut2pmid.pubmed_id
        AND
            medline.pubmed_id = ut2pmid.pubmed_id           
        AND 
            ut2pmid.ut = citations.ut
        AND
            ut2pmid.ambiguous_pmid_2_ut = 0
        AND
            pubmed2taxon.taxon_ncbi = 9606
        AND
            (
                medline.atype_case_reports = 1
            OR
                medline.atype_classical_article = 1
            OR
                medline.atype_clinical_trial = 1
            OR
                medline.atype_clinical_trial_phase_i = 1
            OR
                medline.atype_clinical_trial_phase_ii = 1
            OR
                medline.atype_clinical_trial_phase_iii = 1
            OR
                medline.atype_clinical_trial_phase_iv = 1
            OR
                medline.atype_comparative_study = 1
            OR
                medline.atype_historical_article = 1
            OR
                medline.atype_journal_article = 1
            OR
                medline.atype_meta_analysis = 1
            OR
                medline.atype_multicenter_study = 1
            OR
                medline.atype_observational_study = 1
            OR
                medline.atype_randomized_controlled_trial = 1
            OR
                medline.atype_twin_study = 1
            OR
                medline.atype_validation_studies = 1
            )
        AND
            medline.atype_review = 0
        AND
            medline.j_name_s IN
                (
                SELECT j_name_s
                FROM type_ratio 
                WHERE type_ratio.atype_review < 0.5
                )
        AND
            ut2pmid.score >= 95
        AND
            ut2pmid.ambiguous_pmid_2_ut = 0
'''

In [3]:
%%time
# first import (relatively slow)
df = medline.sql_medline_wos(sql)

CPU times: user 29.1 s, sys: 12.9 s, total: 42.1 s
Wall time: 1min 10s


In [4]:
%%time
# second import (relatively fast)
df = medline.sql_medline_wos(sql)


            Retreive data from local cach rather than
            querying database. Set optional paramter
            refresh=True to reload data from database.
            
CPU times: user 360 ms, sys: 615 ms, total: 975 ms
Wall time: 975 ms


In [5]:
df.head()

,pubmed_id,pubdate_year,amount_of_authors,index,ut,citation_year_1933,citation_year_1934,citation_year_1935,citation_year_1936,citation_year_1937,...,citation_year_2008,citation_year_2009,citation_year_2010,citation_year_2011,citation_year_2012,citation_year_2013,citation_year_2014,citation_year_2015,citation_year_2016,citation_year_2017
0,2591067,1989,3.0,27518,A1989CD38000003,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,3458201,1986,3.0,37099,A1986C010700018,0,0,0,0,0,...,1,1,1,4,2,3,2,1,7,0
2,3610142,1987,3.0,38284,A1987H778600001,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,12477932,2002,84.0,224344,000180101600071,0,0,0,0,0,...,100,78,60,63,78,64,41,43,36,13
4,14702039,2004,157.0,264972,000187666800012,0,0,0,0,0,...,47,30,21,31,20,29,32,30,30,15
